In [73]:
import torch
import gc
import os

# Clear GPU memory
def clear_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
# Call the function
clear_gpu_memory()

# Set memory optimization environment variables
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [1]:
# Check for GPU and set device
import torch
if torch.cuda.is_available():
    device = 'cuda'
    print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
    device = 'cpu'
    print('GPU not available, using CPU.')
# Use the 'device' variable when loading models.

GPU not available, using CPU.


In [75]:
   from dotenv import load_dotenv
   import os

   load_dotenv("HF.config")
   hf_token = os.getenv("HF_TOKEN")

In [76]:
from huggingface_hub import  login

#authenticating HF login, as CV17 dataset requires it
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [77]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

'''
# top commented out as anything from now commented out was attempted with common voice, but pivoted to smaller dataset
#columns_to_remove = ['client_id', 'path', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant']

#streaming so no fill download, spanish full dataset is 48 GB!
ds_train = load_dataset(
    "mozilla-foundation/common_voice_6_0",
    "es",
    split="train",
    streaming=True,
    trust_remote_code=True
).remove_columns(columns_to_remove)

ds_test = load_dataset(
    "mozilla-foundation/common_voice_6_0",
    "es",
    split="test",
    streaming=True,
    trust_remote_code=True
).remove_columns(columns_to_remove)

'''
ds_fleur = DatasetDict()
columns_to_keep = ['audio', 'transcription']  # These are the essential columns for speech recognition

# Load Latin American Spanish data. If need European spanish in future, voxpopuli has almost all europ samples
ds_fleur["train"] = load_dataset(
    "google/fleurs",
    "es_419",
    split="train",
    trust_remote_code=True
).select_columns(columns_to_keep)

ds_fleur["test"] = load_dataset(
    "google/fleurs",
    "es_419",
    split="test",
    trust_remote_code=True
).select_columns(columns_to_keep)

print("\nLatin American Spanish sample:")
#print(ds_fleur["train"][0])

print("\nDataset sizes:")
#print(f"Latin American Spanish train set: {len(list(ds_fleur["train"]))} samples")
#print(f"Latin American Spanish test set: {len(list(ds_fleur["test"]))} samples")




Latin American Spanish sample:

Dataset sizes:


In [78]:
from transformers import WhisperFeatureExtractor

#load feature extractor from pre-trained checkpoint
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [79]:
from transformers import WhisperTokenizer

#load tokenizer, has very extensive byte-pair training
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")


In [80]:
input_str = ds_fleur["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

#verifying tokenizer works

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 los murales o garabatos indeseados reciben el nombre de grafiti
Decoded w/ special:    <|startoftranscript|><|es|><|transcribe|><|notimestamps|>los murales o garabatos indeseados reciben el nombre de grafiti<|endoftext|>
Decoded w/out special: los murales o garabatos indeseados reciben el nombre de grafiti
Are equal:             True


In [81]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="spanish", task="transcribe")

In [82]:
print(ds_fleur["train"][0])


{'audio': {'path': 'train/10005668950815513748.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'transcription': 'los murales o garabatos indeseados reciben el nombre de grafiti'}


In [83]:
from datasets import Audio

#downsample audio to 16kHz to match that of Whisper's sampling rate
ds_fleur = ds_fleur.cast_column("audio", Audio(sampling_rate=16000))


In [84]:
print(ds_fleur["train"][0])


{'audio': {'path': 'train/10005668950815513748.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'transcription': 'los murales o garabatos indeseados reciben el nombre de grafiti'}


In [85]:
def prepare_dataset(batch, feature_extractor=feature_extractor, tokenizer=tokenizer):   
# resample audio to 16kHz
    audio = batch["audio"]
    
    # compute log-mel input feats from arrau
    inputs = feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["input_features"] = inputs.input_features[0]
    batch["attention_mask"] = inputs.attention_mask[0]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch


In [86]:
ds_fleur = ds_fleur.map(
    prepare_dataset,
    remove_columns=ds_fleur.column_names["train"],
    num_proc=4,  # Process 16 samples at a time 
    fn_kwargs={"feature_extractor": feature_extractor, "tokenizer": tokenizer}
)


Map (num_proc=4):   0%|          | 0/2796 [00:00<?, ? examples/s]

AttributeError: 'numpy.ndarray' object has no attribute 'input_features'

In [ ]:
from transformers import WhisperForConditionalGeneration

# gen text conditioned on an input(audio data)
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [ ]:
model.generation_config.language = "spanish"
model.generation_config.task = "transcribe"

model.config.pad_token_id = model.config.eos_token_id


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass 
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int
    # called when collator is used to batch samples together
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        
        # Extract audio features from each sample and pad them to same length
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        #uses processor  built earlier that can call extractor and tokenizer
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Extract text labels from each sample and pad them to same length
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding tokens with -100 (which is ignored in loss calculation)
        #when labels attention mask does NotEqual (ne) 1, then returns True. When this is true, meaning padding token,
        #then it will replace it with a -100
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100) 

        # Remove the start token if it was added during tokenization
        # (it will be added again during generation)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        #batch with audio feats and labels
        return batch


In [ ]:
#initialize collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")


In [ ]:
import numpy as np
'''
- replaces -100 with `pad_token_id` in `label_ids`
    - then decodes predicted and label ids to string
    - Computes WER
'''

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = np.array(pred.label_ids)

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    #two different lines because compute returns dict[Unknown, Unknown] or None. So, first need to get wer val to * by 100
    #but before run won't let multiply by None, so need to check if None or a real value
    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": 100 * wer}


In [ ]:
from transformers.training_args_seq2seq import Seq2SeqTrainingArguments

#define arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-es",  # change to a repo name of your choice
    run_name="whisper-spanish-training",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=200,
    max_steps=2500,
    gradient_checkpointing=True,
    fp16=True,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=50,
    report_to=["tensorboard","wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    eval_strategy="steps",
    save_strategy="steps"
)


In [ ]:
# Clear memory before training
clear_gpu_memory()

In [ ]:
import torch
import gc
'''
# Clear GPU memory
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
'''

In [ ]:
from transformers.trainer_seq2seq import Seq2SeqTrainer
from torchdata.stateful_dataloader import StatefulDataLoader

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds_fleur["train"],
    eval_dataset=ds_fleur["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("HF.config")
wandb_key = os.getenv("WANDB_API_KEY")
wandb_nb_name = os.getenv("WANDB_NOTEBOOK_NAME")

trainer.train()